In [1]:
import sys
sys.path.append('..')
import edlib
import numpy as np
from collections import Counter, defaultdict
import operator
from string import ascii_uppercase


from lrd_parser import LRD_Report
from utils.bio import hamming_distance, identity_shift, OverlapAlignment, compress_homopolymer
import networkx as nx

import matplotlib
%matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

from ndex2.nice_cx_network import NiceCXNetwork
import ndex2.client as nc
import ndex2

%load_ext autoreload
%autoreload 2

In [45]:
lrd_report_fn = '/Poppy/abzikadze/centroFlye/centroFlye_repo/experiments/20191023/lrd_d6z1_rel3_Karen/decomposition.tsv'
monomers_fn = "/Poppy/abzikadze/tandem_flye/data/human/isolated_centromeres/extracted_HORs/CEN6/monomers/inferred_monomers_single.fa"

lrd_report = LRD_Report(lrd_report_fn=lrd_report_fn, monomers_fn=monomers_fn)

In [46]:
monomer_strings = {r_id: record.string for r_id, record in lrd_report.records.items()}

In [47]:
def filter_strings(monomer_strings, max_gap=0.05):
    filtered_strings = {}
    for r_id, string in monomer_strings.items():
        ngaps = Counter(string)['=']
        if ngaps / len(string) <= max_gap:
            filtered_strings[r_id] = string
    return filtered_strings

In [99]:
monomer_strings = filter_strings(monomer_strings)

In [100]:
def get_frequent_kmers(monomer_strings, k, min_mult=10):
    all_kmers = Counter()
    read_kmer_locations = defaultdict(list)
    for r_id, string in monomer_strings.items():
        for i in range(len(string)-k+1):
            kmer = string[i:i+k]
            if '=' not in kmer:
                all_kmers[kmer] += 1
                read_kmer_locations[kmer].append((r_id, i))
                
    frequent_kmers = {kmer: cnt for kmer, cnt in all_kmers.items()
                      if cnt >= min_mult}
    frequent_kmers_read_pos = {kmer: read_kmer_locations[kmer] for kmer in frequent_kmers}
    return frequent_kmers, frequent_kmers_read_pos


In [127]:
k=200
frequent_kmers, frequent_kmers_read_pos = get_frequent_kmers(monomer_strings, k=k)

In [128]:
class DeBruijnGraph:
    def __init__(self, k):
        self.graph = nx.DiGraph()
        self.k = k

    def add_kmer(self, kmer, coverage=1):
        self.graph.add_edge(kmer[:-1], kmer[1:],
                            edge_kmer=kmer,
                            coverages=[coverage])

    def add_kmers(self, kmers, coverage=None):
        for kmer in kmers:
            if coverage is None:
                self.add_kmer(kmer)
            else:
                self.add_kmer(kmer, coverage=coverage[kmer])
                
    def collapse_nonbranching_paths(self):
        def node_on_nonbranching_path(graph, node):
            return nx.number_of_nodes(graph) > 1 \
                and graph.in_degree(node) == 1 \
                and graph.out_degree(node) == 1

        for node in list(self.graph.nodes()):
            if node_on_nonbranching_path(self.graph, node):
                in_edge = list(self.graph.in_edges(node))[0]
                out_edge = list(self.graph.out_edges(node))[0]
                # in_edge_color = self.graph.edges[in_edge]['color']
                # out_edge_color = self.graph.edges[out_edge]['color']
                in_edge_kmer = self.graph.edges[in_edge]['edge_kmer']
                out_edge_kmer = self.graph.edges[out_edge]['edge_kmer']
                in_edge_cov = self.graph.edges[in_edge]['coverages']
                out_edge_cov = self.graph.edges[out_edge]['coverages']

                in_node = in_edge[0]
                out_node = out_edge[1]

                new_kmer = in_edge_kmer + \
                    out_edge_kmer[-(len(out_edge_kmer)-self.k+1):]
                new_coverages = in_edge_cov + out_edge_cov
                new_coverages.sort()
                self.graph.add_edge(in_node, out_node,
                                    edge_kmer=new_kmer,
                                    # color=in_edge_color,
                                    coverages=new_coverages,
                                    length=self.k+len(new_coverages)-1)
                self.graph.remove_node(node)



In [129]:
db = DeBruijnGraph(k=k)
db.add_kmers(frequent_kmers, coverage=frequent_kmers)

In [130]:
db.collapse_nonbranching_paths()
edge_lens = []
for edge in db.graph.edges():
    edge_lens.append((len(db.graph.get_edge_data(*edge)['edge_kmer']),
                      np.median(db.graph.get_edge_data(*edge)['coverages'])))

edge_lens.sort()
print(edge_lens)

[(200, 10.0), (200, 13.0), (200, 32.0), (200, 1624.0), (201, 24.0), (201, 28.0), (201, 36.0), (202, 10.0), (202, 27.0), (202, 1613.0), (202, 1750.0), (204, 10.0), (204, 32.0), (204, 33.0), (204, 37.0), (204, 39.0), (204, 45.0), (204, 50.0), (206, 15.0), (206, 16.0), (206, 47.0), (206, 64.0), (206, 87.0), (207, 10.0), (207, 33.0), (207, 79.5), (207, 93.0), (207, 104.0), (207, 126.0), (207, 137.0), (209, 10.0), (209, 12.0), (209, 13.0), (209, 13.0), (209, 13.0), (209, 15.5), (209, 16.0), (209, 16.0), (209, 16.0), (209, 23.0), (209, 25.0), (209, 25.0), (209, 26.0), (209, 55.5), (209, 65.0), (209, 66.0), (209, 66.0), (209, 112.5), (210, 61.0), (210, 1750.0), (212, 11.0), (212, 13.0), (212, 16.0), (212, 16.0), (212, 17.0), (212, 20.0), (212, 29.0), (213, 26.0), (213, 31.0), (214, 14.0), (214, 26.0), (214, 40.0), (214, 42.0), (214, 48.0), (214, 127.0), (216, 24.0), (216, 25.0), (216, 37.0), (217, 25.0), (217, 39.0), (217, 76.0), (217, 84.0), (217, 98.0), (218, 10.0), (219, 25.0), (219, 32.5)

In [123]:
nx.number_weakly_connected_components(db.graph)

8

In [124]:
nice_cx_debr_graph = ndex2.create_nice_cx_from_networkx(db.graph)

nice_cx_debr_graph.upload_to(server='public.ndexbio.org', username = 'seryrzu',
                             password = 'Kxoq)V?Z]vrgt87x*XO,:we)U&RwEEG!')

Generating CX


'http://public.ndexbio.org/v2/network/c9c6c60d-fc24-11e9-bb65-0ac135e8bacf'

In [68]:
cclens = []

for cc in nx.weakly_connected_components(db.graph):
    print(len(cc))
    cclens.append(len(cc))
cclens.sort()
print()

2333
21
13
62
15
252
14
33
5
126
64
255
281
4
49
125
245
5
98
20
108
238
132
268
14
16
54
30
38
12
15
32
45
3
11
121
33
131
15
497
222
49
5
2
9
213
43
18
22
11
85
41
3
9


In [9]:
def get_read_coverage(frequent_kmers_read_pos, monomer_strings, k):
    coverage = {}
    for r_id, string in monomer_strings.items():
        coverage[r_id] = [0] * (len(string) + 1)
    for pairs in frequent_kmers_read_pos.values():
        for r_id, pos in pairs:
            coverage[r_id][pos] += 1
            coverage[r_id][pos+k] -= 1
    for r_id in coverage:
        coverage[r_id] = np.cumsum(coverage[r_id])
        coverage[r_id] = coverage[r_id][:-1]
    return coverage

In [655]:
# coverage = get_read_coverage(frequent_kmers_read_pos=frequent_kmers_read_pos,
#                              monomer_strings=monomer_strings,
#                              k=k)

In [10]:
# r_id = '0c0dd3f7-6828-447b-ac53-1d118b0270a0'
# r_id = '04203cad-4a32-4b9c-b7dd-c8603e54b46b'
# r_id = '05a4fa6a-38b1-40d6-916e-886ad1fb050b'
# r_id = "aff6bdf9-4388-46e6-9aa5-af5c6c88258d"
# r_id = "182f1138-55b1-45dc-8230-e052f3aa6a44"
# r_id = "054f1983-9204-4889-936a-64ab45e7810c"
# r_id = "1d0706f6-cc39-4a05-a4fd-2ee7e1674457"
# r_id = "46a4165e-6315-4339-9d4e-3af1ca00b3fc"
# plt.plot(coverage[r_id])

In [11]:
# lrd_report.records[r_id].string

In [12]:
def find_zero_cov(coverage):
    all_zero_cov = {}
    for r_id in coverage:
        zero_cov_flatten = np.where(coverage[r_id] == 0)[0]
        if len(zero_cov_flatten) == 0:
            all_zero_cov[r_id] = []
            continue
        zero_cov = []
        zero_cov.append([zero_cov_flatten[0]])
        for pos in zero_cov_flatten[1:]:
            if pos == zero_cov[-1][-1] + 1:
                zero_cov[-1].append(pos)
            else:
                zero_cov.append([pos])
        
        all_zero_cov[r_id] = [(x[0], x[-1]) for x in zero_cov]
        
    return all_zero_cov

In [13]:
# zero_cov = find_zero_cov(coverage=coverage)

In [20]:
def find_path_debr(zero_cov, read_seq, r_id, k, db,
                   max_len=1000, min_len=1, min_overlap=5, max_overlap=30, max_dist=3):
    k -= 1
    results = {}
    corrected_seq = list(read_seq)
    for st, en in zero_cov[::-1]:
        if st < k or en > len(read_seq) - 1 - k or en-st+1 > max_len or en-st+1 < min_len:
            # results[(st, en)] = '-'
            continue
        # print(r_id, st, en)
        st_kmer, en_kmer = read_seq[st-k:st], read_seq[en+1:en+1+k]
        # assert st_kmer in frequent_kmers
        # assert en_kmer in frequent_kmers
        # assert read_seq[st-k+1:st+1] not in frequent_kmers
        # assert read_seq[en:en+k] not in frequent_kmers
        # assert st_kmer in db.graph.nodes

        u = st_kmer
        kmers = [u]
        while u != en_kmer and len(kmers) < 2*k:
            out_edges = list(db.graph.out_edges(u))
            if len(out_edges) == 1:
                edge = out_edges[0]
                assert edge[0] == u
                u = edge[1]
                kmers.append(u)
            else:
                break
        
        if len(kmers) < min_overlap + 1:
            # results[(st, en)] = '-'
            continue

        # print(len(kmers))
        extension = [kmer[-1] for kmer in kmers[1:]]
        extension = ''.join(extension)
        read_segment = read_seq[en+1:en+len(extension)]
        
        ident = identity_shift(extension[:max_overlap],
                               read_segment[:max_overlap],
                               min_overlap=min_overlap,
                               match_char=set('='))
        if ident['id'] == 1 and ident['shift'] is not None:
            # print(ident)
            correction = extension[:ident['shift']]
            if abs(len(correction) - (en-st+1)) > 10:
                # results[(st, en)] = '-'
                continue
            # print(read_seq[st:en+1], read_seq[st-5:en+6])
            # print(correction, len(correction))
            # print("")
            results[(st, en)] = (read_seq[st:en+1], correction)
            corrected_seq[st:en+1] = list(correction)
    corrected_seq = ''.join(corrected_seq)
    return results, corrected_seq
            
            

In [21]:
# all_res = []
# for r_id in zero_cov:
#     results, corrected_seq = find_path_debr(zero_cov[r_id], monomer_strings[r_id], r_id=r_id, k=k, db=db)
#     all_res += list(results.values())
# all_res = np.array(all_res)

In [22]:
def correct_seq(monomer_strings, k, min_mult=10):
    frequent_kmers, frequent_kmers_read_pos = get_frequent_kmers(monomer_strings, k=k, min_mult=min_mult)
    db = DeBruijnGraph(k=k)
    db.add_kmers(frequent_kmers, coverage=frequent_kmers)
    coverage = get_read_coverage(frequent_kmers_read_pos=frequent_kmers_read_pos,
                                 monomer_strings=monomer_strings,
                                 k=k)
    zero_cov = find_zero_cov(coverage=coverage)
    
    all_corrections = {}
    corrected_seqs = {}
    for r_id in zero_cov:
        all_corrections[r_id], corrected_seqs[r_id] = find_path_debr(zero_cov[r_id],
                                                              monomer_strings[r_id],
                                                              r_id=r_id,
                                                              k=k,
                                                              db=db)
    return all_corrections, corrected_seqs
    

In [62]:
def get_ngaps(strings):
    ngaps = 0
    for r_id, string in strings.items():
        r_ngaps = Counter(string)['=']
        if r_ngaps >= 20:
            mstring = string.lower()
            mstring = mstring.replace('=', '|')
            print(mstring)
            print("")
        ngaps += r_ngaps
    return ngaps

In [49]:
def correct_reads(monomer_strings, min_k=30, max_k=200, niter=1):
    corrected_seqs = monomer_strings
    for k in range(min_k, max_k):
        for i in range(niter):
            print(k, i)
            all_corrections, corrected_seqs = correct_seq(corrected_seqs, k=k)
        for r_id, corrections in all_corrections.items():
            if len(corrections):
                print(r_id, corrections)
    return corrected_seqs

In [50]:
corrected_seqs = correct_reads(monomer_strings)

30 0
0352811e-b17c-4fc1-a73f-99757d30745c {(445, 445): ('=', 'G'), (332, 332): ('=', 'H')}
04203cad-4a32-4b9c-b7dd-c8603e54b46b {(509, 509): ('=', 'E')}
0c0dd3f7-6828-447b-ac53-1d118b0270a0 {(457, 457): ('=', 'J')}
0c20902b-e7d8-44ea-9edf-179fc2b83208 {(667, 667): ('=', 'H')}
0c849de1-fbde-4ff4-bf9e-a7675a011e11 {(385, 385): ('=', 'J')}
10b78685-fe40-49f4-bb76-848371912dea {(517, 517): ('=', 'K'), (119, 119): ('=', 'I')}
14eb3993-37ae-468e-9e48-c2eeeb996ed3 {(389, 389): ('=', 'H')}
15d5e998-80af-4e18-b530-2abde41195de {(136, 136): ('=', 'G')}
16b13a10-65ce-4a43-a8aa-b385ae435bce {(169, 169): ('=', 'I')}
1d76a775-8e2b-4f44-a979-68861b6f25de {(162, 162): ('=', 'J')}
1df6a0f7-28ca-432e-b09c-15f7d61639e0 {(832, 832): ('=', 'J')}
238e11c1-f034-41da-9354-43126513f191 {(582, 582): ('=', 'G'), (193, 193): ('=', 'K')}
25c395b8-2d54-42bc-8378-2298021fa871 {(131, 131): ('=', 'H')}
288dc332-ae38-45ee-8fa6-8a9616d28aaa {(676, 677): ('==', 'IJ')}
2de12301-635c-4a93-a772-42e39317276b {(409, 409): ('=

a61180df-982c-4d59-b8b3-eedad055ac2e {(239, 239): ('D', 'G')}
c774cf4a-14a9-4432-9992-53fa6813189f {(183, 184): ('==', 'GH')}
52 0
0c20902b-e7d8-44ea-9edf-179fc2b83208 {(463, 469): ('=IJK===', 'HIJKLMN')}
e177ca03-2115-4ba8-b1a5-f642bf02a597 {(332, 332): ('=', 'B')}
e271f539-e586-4ca0-b153-01c72400eaa3 {(464, 465): ('==', 'KL')}
fa2fc652-434e-49f5-9386-79109c77c2f6 {(152, 153): ('==', 'MN')}
53 0
54 0
55 0
56 0
57 0
58 0
057047e6-eb15-4d39-b4f6-473ecba0558a {(258, 259): ('==', 'EF')}
063b3d2c-c9b3-4292-861f-2839fc294b2b {(348, 348): ('=', 'F')}
38549dfb-a56b-4665-a5f8-3f68fafa4cdd {(285, 285): ('=', 'H')}
3ca9bb28-d5dc-4081-939b-3da4f258f0d0 {(288, 291): ('=FG=', 'EFGH')}
3f3090f9-9551-44c9-9b6f-430fb93af6c4 {(718, 718): ('=', 'H')}
63fa9a7e-2c00-49fb-905d-27fea8181b9e {(85, 85): ('=', 'F')}
aff6bdf9-4388-46e6-9aa5-af5c6c88258d {(1535, 1536): ('==', 'FG')}
cde82355-e4c9-464c-a614-81cdb3d76664 {(515, 519): ('==HI=', 'FGHIJ')}
dccab6e2-af28-4920-adc4-46518380d7ba {(897, 897): ('=', 'G')}

169 0
170 0
050b88c4-e343-431d-99dc-4b8151b4adef {(376, 378): ('===', 'NOP')}
171 0
172 0
173 0
d84233c4-af6b-45a7-b6f9-1ec9627419dc {(1481, 1482): ('==', 'AB')}
f9347a59-21fa-46f8-b66b-574ad0f95668 {(1143, 1144): ('==', 'RA')}
174 0
175 0
176 0
96779c5c-4eee-4184-ae1c-182e6b5ccb6a {(564, 567): ('====', 'NOPQ')}
cb0d28b5-f911-4971-a214-1e5450f8689c {(778, 778): ('=', 'R')}
dccab6e2-af28-4920-adc4-46518380d7ba {(457, 457): ('=', 'MN')}
fa2fc652-434e-49f5-9386-79109c77c2f6 {(350, 353): ('====', 'ABFG')}
177 0
178 0
1de6e2f4-1900-438b-9239-2ea85e25bd1a {(413, 413): ('C', 'F')}
2150aca2-413b-4207-ab2d-ac2f390a625d {(586, 586): ('C', 'F')}
46a4165e-6315-4339-9d4e-3af1ca00b3fc {(316, 316): ('C', 'F')}
814fadd3-b302-42f2-b650-d53315b7844e {(574, 574): ('C', 'F')}
b55aea96-cbe9-4c7b-9b0a-0501dd830ba8 {(618, 618): ('C', 'F')}
e98f9879-3e90-4503-a35e-54d944e03f98 {(1327, 1327): ('C', 'F')}
f9774a7d-75d0-4783-8053-ce259920fc2e {(342, 342): ('C', 'F')}
179 0
180 0
bef37cd6-6a81-4b50-bc5a-c45d7fc70

In [51]:
get_ngaps(monomer_strings), get_ngaps(corrected_seqs)

(1899, 1448)

In [52]:
# corrected_seqs = correct_reads(corrected_seqs)

In [63]:
get_ngaps(corrected_seqs)

mnopqrabcdefghijklmnopqrabcdefghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabfghijklmnopqrabfghijklmnopqrabfghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabfghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabfghijklmnopqrabfghijklmnopqrabfghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabfghijklmnopqrabfghijklmnopqrabfghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabcd||||||||||||||||laobala|l|lh||||||||kl

ghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabcdefghijklmnopqrabcdefgabfghijklmnopqrabfghijklmnopqrabcdefghijklmnopqrabcdefghij

1448